In [45]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline   

## Change elements below for each experiment

In [46]:
expt='All_TLstables'

wells=['Condition1','Condition2','Condition3','Condition4','Condition5','Condition6','Condition7' ]

colors=['#ef5847','#2993ff','#64abff','#25d1f6','#71f7ff','#ffba0c','#aaaaaa']
# g, d, dmso, asun: '#ffba0c','#ef5847','#aaaaaa','#a39cd3'

inpath='Inpath_here'


font = {'family' : 'Arial',
        'weight' : 'normal',
        'size'   : 16}

matplotlib.rc('font', **font)


## Read in each well for one stat and normalize each cell

In [47]:
def normStat(stat):    
    allwellsonestat=[]
    for w in wells:
        filename=inpath+w+'_'+stat+'.csv'
        statdf=pd.read_csv(filename)
        statdf=statdf.iloc[:,1:]   #get rid of extra index column
        
        for cell, data in statdf.iteritems():   #iterate over columns          
            base=data.iloc[0]     #normalize to first frame
            data=data/base
            statdf[cell]=data
                
            
        allwellsonestat.append(statdf)   #use list to keep in order of well
        
    return allwellsonestat
        
    

## Plot all wells for each stat

In [48]:

time_index=[-8,-6,-4,-2,0,2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38,40,42,44,46,48,50,52,54,56,58,60]
xlabels=[-8, 0, 10, 20, 30, 40, 50, 60]

for s in range(6):

    stats=['Area','Perim','Circ','AR','Round','Solidity']

    allwells=normStat(stats[s])  
    
    ###DMSO subtract
    dmsomean=allwells[6].mean(axis=1)
    dmsostd=allwells[6].sem(axis=1)
    dmsomean.index=time_index
    dmsostd.index=time_index   

    for i, a in enumerate(allwells):
        
        wellmean=a.mean(axis=1)   #mean across columns
        #wellstd=a.std(axis=1)
        wellstd=a.sem(axis=1)         ###SEM
        wellmean.index=time_index
        wellstd.index=time_index
        
        ###DMSO subtraction
        wellmean=wellmean-dmsomean
        inside=wellstd.apply(np.square)+dmsostd.apply(np.square)
        wellstd=inside.apply(np.sqrt)
        
        numcells=len(a.columns)
        #print wells[i]+' '+str(numcells)
        
        if i!=6:
            plt.axhline(y=0, color='0.5', linewidth=0.5, zorder=0 )
            plt.plot(wellmean, color=colors[i], linewidth=3, label=wells[i])
            ytitle=stats[s]+' (baseline normalized)'
            plt.ylabel(ytitle)
            #plt.legend(loc='upper left', fontsize=12)
            plt.fill_between(wellstd.index.values, wellmean-wellstd, wellmean+wellstd, alpha=0.1, edgecolor='None', color=colors[i])
            plt.xticks(xlabels)
            plt.xlabel('Time (min)')
            plt.axvline(x=0, linewidth=0.5, color='0.5')
            plt.xlim([-8,60])
            
            #Enforce same y limits
            if s==0:
                plt.ylim([-0.10, 0.15])
            if s==1:
                plt.ylim([-0.20, 0.15])
            if s==2:
                plt.ylim([-0.1, 0.40])
            if s==5:
                plt.ylim([-0.05, 0.15])
            
            outpath='Outpath_here'
            fig = plt.gcf()
            fig.set_size_inches(12,3.8)
            plt.tight_layout()
            plt.savefig(outpath+expt+'_'+stats[s]+'.png', dpi=200)
        
    plt.clf()   



